In [1]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from itertools import product
from sklearn.model_selection import train_test_split
from pathlib import Path
import random
from tqdm import tqdm
import xml.etree.ElementTree as ET
import cv2
import plotly.graph_objs as go
from torchvision.transforms import Compose
from torchvision.transforms import ToTensor
import json
from torch.utils.data import Dataset
import torch
import cv2
import torch.nn as nn
import torch.nn.functional as F

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


im_size = (252, 252)


# dataset definition
class thyroidDataset(Dataset):
    def __init__(self, split):
        self.all_data = []
        self.compositions = {'Unknown':0, 'cystic':1,
                             'predominantly solid':2,
                             'solid':3, 'spongiform appareance':4}
        self.echogenicities = {'Unknown':0, 'hyperechogenecity':1,
                             'hypoechogenecity':2, 'isoechogenicity':3,
                             'marked hypoechogenecity':4}
        self.margins = {'Unknown':0, 'ill- defined':1, 'microlobulated':2,
                        'spiculated':3, 'well defined smooth':4}
        self.calcifications = {'macrocalcification':0, 'microcalcification':1, 'non':2}
        self.types ={'benign':0, 'malign':1}
        self.types_count = []
        for t_type in ['benign', 'malign']:
            root_dir=Path('../data/' + split + '/' + t_type).expanduser().resolve().absolute() 
            print(root_dir)
            files = list(root_dir.glob("*"))
            labels = [self.types[t_type]] * len(files)
            self.types_count.append(len(files))
            data_list = list(zip(files, labels))
            self.all_data.extend(data_list)
        random.shuffle(self.all_data)
        self.cases, self.types = zip(*self.all_data)
        print("number of data items:" + str(len(self.cases)))
        self.sample_weights = [1/self.types_count[label] for label in self.types]
    def __len__(self):
        return len(self.cases)
  
    def __getitem__(self, idx):
        labels = np.zeros(15, dtype = float)
        xml_data = ET.parse(list(self.cases[idx].glob('*[0-9].xml'))[0]).getroot()
        for x in xml_data:
            if x.tag=='composition' and x.text is not None:
                composition = x.text
                labels[self.compositions[composition] - 1] = 1.0
            if x.tag=='echogenicity' and x.text is not None:
                echogenicity = x.text
                labels[self.echogenicities[echogenicity] + 3] = 1.0
            if x.tag=='margins' and x.text is not None:
                margin = x.text
                labels[self.margins[margin] + 7] = 1.0
            if x.tag=='calcifications' and x.text is not None:
                calcification = x.text
                labels[self.calcifications[calcification] + 11] = 1.0
        xml_data = ET.parse(list(self.cases[idx].glob('*[0-9].xml'))[0]).find("mark")
        for x in xml_data:
            if(x.tag=='svg'):
                encoded = str(x.text)
                poly_data = json.loads(x.text)
        
        #labels[15] = list(self.types)[idx]
        im_name = list(self.cases[idx].glob('*[0-9].jpg'))[0]
        im = cv2.imread(str(im_name))[:, :, 0]
        mask = np.zeros(np.shape(im))
        im = cv2.resize(im, dsize=im_size, interpolation=cv2.INTER_CUBIC)
        
        # add mask 
        for polygon in poly_data:
            xs = []
            ys = []
            for point in polygon["points"]:
                xs.append(point["x"])
                ys.append(point["y"])
            contour = np.concatenate((np.expand_dims(xs, 1), np.expand_dims(ys, 1)), axis=1)
            cv2.fillPoly(mask, pts = [contour], color =(1, 1, 1))
        
        #mask = cv2.resize(mask, dsize=(300, 300), interpolation=cv2.INTER_LINEAR)
        mask = cv2.resize(mask, dsize=im_size, interpolation=cv2.INTER_LINEAR)
        
        
        
        # Adding data augmentation to avoid overfitting
        if random.randint(1, 10) > 5:
            im = np.flipud(im)
        if random.randint(1, 10) > 5:
            im = np.fliplr(im)
        if random.randint(1, 10) > 5:
            for i in range(random.randint(1, 4)):
                im = np.rot90(im)
        im = np.ascontiguousarray(im)

        #plt.figure()
        #plt.imshow(im)

        transforms = Compose([ToTensor()])
        mask = transforms(mask)
        im = transforms(im)
        
        im = im * mask
        
        im = im.type(torch.FloatTensor)
        
        sample = {"image": im, "labels": torch.from_numpy(labels), "types" : self.types[idx], "name": str(im_name)}
        return sample

In [3]:
# Dataset creation
training_set = thyroidDataset(split='train')
parameters_train = {
    "batch_size": 32,
    #"shuffle": True,
}

parameters_test = {
    "batch_size": 1,
    "shuffle": False,
}
training_set = thyroidDataset(split='train')
training_generator = torch.utils.data.DataLoader(training_set, **parameters_train, sampler=torch.utils.data.WeightedRandomSampler(training_set.sample_weights, len(training_set.cases), replacement=True))

training_generator1 = torch.utils.data.DataLoader(training_set, **parameters_test, sampler=torch.utils.data.WeightedRandomSampler(training_set.sample_weights, len(training_set.cases), replacement=True))


testing_set = thyroidDataset(split='test')
testing_generator = torch.utils.data.DataLoader(testing_set, **parameters_test, sampler=torch.utils.data.WeightedRandomSampler(testing_set.sample_weights, len(testing_set.cases), replacement=True))


import torch.distributions

/home/ahana/thyroid/data/train/benign
/home/ahana/thyroid/data/train/malign
number of data items:73
/home/ahana/thyroid/data/train/benign
/home/ahana/thyroid/data/train/malign
number of data items:73
/home/ahana/thyroid/data/test/benign
/home/ahana/thyroid/data/test/malign
number of data items:25


In [4]:
class Encoder(nn.Module):
    
    def __init__(self, encoded_space_dim,fc2_input_dim):
        super().__init__()
        
        self.N = torch.distributions.Normal(0, 1)

        self.N.loc = self.N.loc.cuda() # hack to get sampling on the GPU
        self.N.scale = self.N.scale.cuda()
        self.kl = 0


        ### Convolutional section
        self.encoder_cnn = nn.Sequential(
            nn.Conv2d(1, 8, 3, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv2d(8, 16, 3, stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.Conv2d(16, 32, 3, stride=2, padding=0),
            nn.ReLU(True),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, 3, stride=2, padding=0),
            nn.ReLU(True),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, 3, stride=2, padding=0),
            nn.ReLU(True),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 256, 3, stride=2, padding=0),
            nn.ReLU(True)

        )
        
        ### Flatten layer
        self.flatten = nn.Flatten(start_dim=1)
### Linear section
        self.encoder_fc = nn.Sequential(
            nn.Linear(2304 + 15, fc2_input_dim * 4),
            nn.ReLU(True),
            nn.Linear(4* fc2_input_dim, 2 * fc2_input_dim)
        )
        self.encoder_lin = nn.Sequential(
            nn.Linear(2 * fc2_input_dim, fc2_input_dim),
            nn.ReLU(True),
            nn.Linear(fc2_input_dim, encoded_space_dim)
        )
        self.encoder_lin1 = nn.Sequential(
            nn.Linear(2 * fc2_input_dim, fc2_input_dim),
            nn.ReLU(True),
            nn.Linear(fc2_input_dim, encoded_space_dim)
        )
        
    def forward(self, x, attributes):
        #print("Encoder input: ", np.shape(x))
        x = self.encoder_cnn(x)
        
        x = self.flatten(x)
        #print("Encoder flattened output: ", np.shape(x))
        #x = self.encoder_lin(x)
        x = torch.cat((x, attributes), dim=1)
        x = self.encoder_fc(x.float())
        mu =  self.encoder_lin(x)
        sigma = torch.exp(self.encoder_lin1(x))
        z = mu + sigma*self.N.sample(mu.shape)
        self.kl = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()

        return z
    
    
    
class Decoder(nn.Module):
    
    def __init__(self, encoded_space_dim,fc2_input_dim):
        super().__init__()
        self.decoder_lin = nn.Sequential(
            nn.Linear(encoded_space_dim + 15, fc2_input_dim),
            nn.ReLU(True),
            nn.Linear(fc2_input_dim, 3 * 3 * fc2_input_dim),
            nn.ReLU(True)
        )

        self.unflatten = nn.Unflatten(dim=1, 
        unflattened_size=(fc2_input_dim, 3, 3))

        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(fc2_input_dim, 128, 3, stride=2, output_padding=0),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 3, stride=2, output_padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, 3, stride=2, output_padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 16, 3, stride=2, output_padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 3, stride=2,  padding=1, output_padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 3, stride=2, padding=1, output_padding=1)
        )
        
    def forward(self, x):
        x = self.decoder_lin(x)
        x = self.unflatten(x)
        
        x = self.decoder_conv(x)
        x = torch.sigmoid(x)
        return x

In [5]:
### Define the loss function
loss_fn = torch.nn.MSELoss()
loss_latent = torch.nn.L1Loss()



### Define an optimizer (both for the encoder and the decoder!)
lr= 0.001

### Set the random seed for reproducible results
torch.manual_seed(0)

### Initialize the two networks
d = 16

#model = Autoencoder(encoded_space_dim=encoded_space_dim)
encoder = Encoder(encoded_space_dim=d,fc2_input_dim=512)
decoder = Decoder(encoded_space_dim=d,fc2_input_dim=512)
params_to_optimize = [
    {'params': encoder.parameters()},
    {'params': decoder.parameters()}
]

optim = torch.optim.Adam(params_to_optimize, lr=lr, weight_decay=1e-05)

# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')

# Move both the encoder and the decoder to the selected device
encoder.to(device)
decoder.to(device)

Selected device: cuda


Decoder(
  (decoder_lin): Sequential(
    (0): Linear(in_features=31, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=4608, bias=True)
    (3): ReLU(inplace=True)
  )
  (unflatten): Unflatten(dim=1, unflattened_size=(512, 3, 3))
  (decoder_conv): Sequential(
    (0): ConvTranspose2d(512, 128, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(2, 2))
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(32, 16, kernel_size=(3, 3), stride=(2, 2))
    (10): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=T

In [6]:
### Training function
def train_epoch(encoder, decoder, device, dataloader, loss_fn, optimizer):
    # Set train mode for both the encoder and the decoder
    encoder.train()
    decoder.train()
    train_loss = []
    # Iterate the dataloader (we do not need the label values, this is unsupervised learning)
    for data in training_generator: # with "_" we just ignore the labels (the second element of the dataloader tuple)
        # Move tensor to the proper device
        image_batch = data["image"]
        image_batch = image_batch.to(device)
        
        
        #print("Random data shape", np.shape(random_data), "; data[labels] ", np.shape(data["labels"]))
        latent = data["labels"]
        latent = latent.to(device)
        
        #print(latent)
        
        # Encode data
        encoded_data = encoder(image_batch, latent)
        
        # Decode data
        
        decoder_input = torch.cat((encoded_data, latent), dim = 1)
        
        decoded_data = decoder(decoder_input.float())
        # Evaluate loss

        #print("Encoder.kl : ", encoder.kl)

        d = ((image_batch - decoded_data)**2).sum()
        loss = d + 0.5*encoder.kl

        #loss = loss_fn(decoded_data, image_batch) + 0.2*loss_latent(encoded_data, latent.float())
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Print batch loss
        #print('\t partial train loss (single batch): %f' % (loss.data))
        #train_loss.append(loss.detach().cpu().numpy())

    return loss

In [ ]:
num_epochs = 100000
diz_loss = {'train_loss':[],'val_loss':[]}
for epoch in range(num_epochs):
   train_loss = train_epoch(encoder,decoder,device,training_generator,loss_fn,optim)
   print('\n EPOCH {}/{} \t train loss {}'.format(epoch + 1, num_epochs,train_loss))
   diz_loss['train_loss'].append(train_loss)



 EPOCH 1/100000 	 train loss 305662.0

 EPOCH 2/100000 	 train loss 130573.4296875

 EPOCH 3/100000 	 train loss 126032.9921875

 EPOCH 4/100000 	 train loss 120709.3671875

 EPOCH 5/100000 	 train loss 116811.390625

 EPOCH 6/100000 	 train loss 109401.40625

 EPOCH 7/100000 	 train loss 106705.609375

 EPOCH 8/100000 	 train loss 106586.9140625

 EPOCH 9/100000 	 train loss 99921.296875

 EPOCH 10/100000 	 train loss 99998.578125

 EPOCH 11/100000 	 train loss 92383.609375

 EPOCH 12/100000 	 train loss 87242.96875

 EPOCH 13/100000 	 train loss 88669.3359375

 EPOCH 14/100000 	 train loss 86757.109375

 EPOCH 15/100000 	 train loss 81917.2421875

 EPOCH 16/100000 	 train loss 79383.828125

 EPOCH 17/100000 	 train loss 78128.9296875

 EPOCH 18/100000 	 train loss 77092.796875

 EPOCH 19/100000 	 train loss 72473.6875

 EPOCH 20/100000 	 train loss 69547.109375

 EPOCH 21/100000 	 train loss 73111.0625

 EPOCH 22/100000 	 train loss 64137.3203125

 EPOCH 23/100000 	 train loss 65556


 EPOCH 179/100000 	 train loss 3221.1494140625

 EPOCH 180/100000 	 train loss 3378.384521484375

 EPOCH 181/100000 	 train loss 3331.985107421875

 EPOCH 182/100000 	 train loss 3219.158203125

 EPOCH 183/100000 	 train loss 2980.71630859375

 EPOCH 184/100000 	 train loss 3229.03759765625

 EPOCH 185/100000 	 train loss 3113.606201171875

 EPOCH 186/100000 	 train loss 3448.608154296875

 EPOCH 187/100000 	 train loss 3157.03564453125

 EPOCH 188/100000 	 train loss 3204.33837890625

 EPOCH 189/100000 	 train loss 3071.444091796875

 EPOCH 190/100000 	 train loss 3047.21142578125

 EPOCH 191/100000 	 train loss 3289.726806640625

 EPOCH 192/100000 	 train loss 2954.08740234375

 EPOCH 193/100000 	 train loss 3259.030517578125

 EPOCH 194/100000 	 train loss 2946.657470703125

 EPOCH 195/100000 	 train loss 2840.1728515625

 EPOCH 196/100000 	 train loss 2969.073974609375

 EPOCH 197/100000 	 train loss 2996.828857421875

 EPOCH 198/100000 	 train loss 3154.97998046875

 EPOCH 199/10


 EPOCH 346/100000 	 train loss 1542.8267822265625

 EPOCH 347/100000 	 train loss 1293.681640625

 EPOCH 348/100000 	 train loss 1305.8082275390625

 EPOCH 349/100000 	 train loss 1327.851806640625

 EPOCH 350/100000 	 train loss 1516.279296875

 EPOCH 351/100000 	 train loss 1584.8648681640625

 EPOCH 352/100000 	 train loss 1401.1053466796875

 EPOCH 353/100000 	 train loss 1351.260009765625

 EPOCH 354/100000 	 train loss 1251.37646484375

 EPOCH 355/100000 	 train loss 1390.8797607421875

 EPOCH 356/100000 	 train loss 1218.57275390625

 EPOCH 357/100000 	 train loss 1595.8619384765625

 EPOCH 358/100000 	 train loss 1200.2235107421875

 EPOCH 359/100000 	 train loss 1539.3154296875

 EPOCH 360/100000 	 train loss 1403.007080078125

 EPOCH 361/100000 	 train loss 1735.9892578125

 EPOCH 362/100000 	 train loss 1281.897705078125

 EPOCH 363/100000 	 train loss 1391.8033447265625

 EPOCH 364/100000 	 train loss 1245.9107666015625

 EPOCH 365/100000 	 train loss 1135.17822265625

 EP


 EPOCH 511/100000 	 train loss 991.3472900390625

 EPOCH 512/100000 	 train loss 711.0184936523438

 EPOCH 513/100000 	 train loss 826.5079345703125

 EPOCH 514/100000 	 train loss 964.9468383789062

 EPOCH 515/100000 	 train loss 1013.9359741210938

 EPOCH 516/100000 	 train loss 864.642333984375

 EPOCH 517/100000 	 train loss 1152.2469482421875

 EPOCH 518/100000 	 train loss 981.7852783203125

 EPOCH 519/100000 	 train loss 1059.8160400390625

 EPOCH 520/100000 	 train loss 861.8546752929688

 EPOCH 521/100000 	 train loss 750.1403198242188

 EPOCH 522/100000 	 train loss 1142.5897216796875

 EPOCH 523/100000 	 train loss 964.1162109375

 EPOCH 524/100000 	 train loss 881.8485107421875

 EPOCH 525/100000 	 train loss 892.5091552734375

 EPOCH 526/100000 	 train loss 875.576416015625

 EPOCH 527/100000 	 train loss 962.9749145507812

 EPOCH 528/100000 	 train loss 753.8853149414062

 EPOCH 529/100000 	 train loss 959.09619140625

 EPOCH 530/100000 	 train loss 880.1771240234375

 E


 EPOCH 677/100000 	 train loss 622.69873046875

 EPOCH 678/100000 	 train loss 557.4697875976562

 EPOCH 679/100000 	 train loss 630.8005981445312

 EPOCH 680/100000 	 train loss 1154.7835693359375

 EPOCH 681/100000 	 train loss 887.5482788085938

 EPOCH 682/100000 	 train loss 551.1986694335938

 EPOCH 683/100000 	 train loss 627.4716186523438

 EPOCH 684/100000 	 train loss 949.8656005859375

 EPOCH 685/100000 	 train loss 704.0037841796875

 EPOCH 686/100000 	 train loss 699.9603271484375

 EPOCH 687/100000 	 train loss 627.5810546875

 EPOCH 688/100000 	 train loss 637.9986572265625

 EPOCH 689/100000 	 train loss 596.1170043945312

 EPOCH 690/100000 	 train loss 824.1650390625

 EPOCH 691/100000 	 train loss 739.658447265625

 EPOCH 692/100000 	 train loss 631.401611328125

 EPOCH 693/100000 	 train loss 722.1480712890625

 EPOCH 694/100000 	 train loss 803.3535766601562

 EPOCH 695/100000 	 train loss 889.340576171875

 EPOCH 696/100000 	 train loss 626.2576293945312

 EPOCH 69

...


 EPOCH 3769/100000 	 train loss 449.56549072265625

 EPOCH 3770/100000 	 train loss 594.296142578125

 EPOCH 3771/100000 	 train loss 617.17822265625

 EPOCH 3772/100000 	 train loss 979.3893432617188

 EPOCH 3773/100000 	 train loss 547.5576171875

 EPOCH 3774/100000 	 train loss 609.7576904296875

 EPOCH 3775/100000 	 train loss 600.368408203125

 EPOCH 3776/100000 	 train loss 501.4836730957031

 EPOCH 3777/100000 	 train loss 852.98388671875

 EPOCH 3778/100000 	 train loss 516.59765625

 EPOCH 3779/100000 	 train loss 598.0301513671875

 EPOCH 3780/100000 	 train loss 495.67230224609375

 EPOCH 3781/100000 	 train loss 566.4197998046875

 EPOCH 3782/100000 	 train loss 450.817138671875

 EPOCH 3783/100000 	 train loss 391.5381164550781

 EPOCH 3784/100000 	 train loss 602.26220703125

 EPOCH 3785/100000 	 train loss 1071.6077880859375

 EPOCH 3786/100000 	 train loss 609.8865966796875

 EPOCH 3787/100000 	 train loss 349.9799499511719

 EPOCH 3788/100000 	 train loss 1071.3364257


 EPOCH 3931/100000 	 train loss 331.555419921875

 EPOCH 3932/100000 	 train loss 477.1939697265625

 EPOCH 3933/100000 	 train loss 340.26092529296875

 EPOCH 3934/100000 	 train loss 234.693359375

 EPOCH 3935/100000 	 train loss 445.5726013183594

 EPOCH 3936/100000 	 train loss 417.92919921875

 EPOCH 3937/100000 	 train loss 277.05731201171875

 EPOCH 3938/100000 	 train loss 292.1360778808594

 EPOCH 3939/100000 	 train loss 332.225830078125

 EPOCH 3940/100000 	 train loss 353.4809875488281

 EPOCH 3941/100000 	 train loss 418.27972412109375

 EPOCH 3942/100000 	 train loss 295.73638916015625

 EPOCH 3943/100000 	 train loss 343.5893249511719

 EPOCH 3944/100000 	 train loss 418.3992919921875

 EPOCH 3945/100000 	 train loss 355.6845703125

 EPOCH 3946/100000 	 train loss 334.0557556152344

 EPOCH 3947/100000 	 train loss 362.88092041015625

 EPOCH 3948/100000 	 train loss 314.273193359375

 EPOCH 3949/100000 	 train loss 362.369384765625

 EPOCH 3950/100000 	 train loss 378.50


 EPOCH 4093/100000 	 train loss 412.37445068359375

 EPOCH 4094/100000 	 train loss 376.7083435058594

 EPOCH 4095/100000 	 train loss 405.9243469238281

 EPOCH 4096/100000 	 train loss 337.27923583984375

 EPOCH 4097/100000 	 train loss 326.16021728515625

 EPOCH 4098/100000 	 train loss 385.6965637207031

 EPOCH 4099/100000 	 train loss 368.3167724609375

 EPOCH 4100/100000 	 train loss 337.08154296875

 EPOCH 4101/100000 	 train loss 372.5851745605469

 EPOCH 4102/100000 	 train loss 494.1510009765625

 EPOCH 4103/100000 	 train loss 348.4629211425781

 EPOCH 4104/100000 	 train loss 418.87646484375

 EPOCH 4105/100000 	 train loss 343.9559020996094

 EPOCH 4106/100000 	 train loss 349.69952392578125

 EPOCH 4107/100000 	 train loss 293.5696105957031

 EPOCH 4108/100000 	 train loss 278.543701171875

 EPOCH 4109/100000 	 train loss 538.7564697265625

 EPOCH 4110/100000 	 train loss 352.5230712890625

 EPOCH 4111/100000 	 train loss 410.08746337890625

 EPOCH 4112/100000 	 train los


 EPOCH 4254/100000 	 train loss 317.9032287597656

 EPOCH 4255/100000 	 train loss 354.294189453125

 EPOCH 4256/100000 	 train loss 440.91864013671875

 EPOCH 4257/100000 	 train loss 329.2049255371094

 EPOCH 4258/100000 	 train loss 419.46148681640625

 EPOCH 4259/100000 	 train loss 343.89990234375

 EPOCH 4260/100000 	 train loss 289.1492614746094

 EPOCH 4261/100000 	 train loss 321.39984130859375

 EPOCH 4262/100000 	 train loss 399.98614501953125

 EPOCH 4263/100000 	 train loss 267.950439453125

 EPOCH 4264/100000 	 train loss 365.11553955078125

 EPOCH 4265/100000 	 train loss 453.2189025878906

 EPOCH 4266/100000 	 train loss 380.897216796875

 EPOCH 4267/100000 	 train loss 371.8917541503906

 EPOCH 4268/100000 	 train loss 362.8155822753906

 EPOCH 4269/100000 	 train loss 416.0416564941406

 EPOCH 4270/100000 	 train loss 374.61419677734375

 EPOCH 4271/100000 	 train loss 389.6842041015625

 EPOCH 4272/100000 	 train loss 467.3454284667969

 EPOCH 4273/100000 	 train lo


 EPOCH 4416/100000 	 train loss 301.8504638671875

 EPOCH 4417/100000 	 train loss 324.2147216796875

 EPOCH 4418/100000 	 train loss 326.4446105957031

 EPOCH 4419/100000 	 train loss 378.5169372558594

 EPOCH 4420/100000 	 train loss 291.2576904296875

 EPOCH 4421/100000 	 train loss 370.8482666015625

 EPOCH 4422/100000 	 train loss 304.5286865234375

 EPOCH 4423/100000 	 train loss 347.51318359375

 EPOCH 4424/100000 	 train loss 378.71331787109375

 EPOCH 4425/100000 	 train loss 410.9040222167969

 EPOCH 4426/100000 	 train loss 798.3575439453125

 EPOCH 4427/100000 	 train loss 479.4822998046875

 EPOCH 4428/100000 	 train loss 408.77813720703125

 EPOCH 4429/100000 	 train loss 420.5904846191406

 EPOCH 4430/100000 	 train loss 373.214111328125

 EPOCH 4431/100000 	 train loss 311.00689697265625


In [12]:
import torchvision
from torchvision import transforms
import random

def show_image(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

encoder.eval()
decoder.eval()

with torch.no_grad():
    # calculate mean and std of latent code, generated takining in test images as inputs 
    images = torch.from_numpy(np.round(np.random.rand(100, 1, im_size[0], im_size[1]))).float()
    labels = np.zeros(15, dtype = float)
    
    # Setting labels for each attribute randomly based on number of classes for each
    randnum = random.randint(0,3)
    labels[randnum] = 1.0
    
    randnum = random.randint(0,3))
    labels[randnum + 4] = 1.0
    
    randnum = random.randint(0,3))
    labels[randnum + 8] = 1.0
    
    randnum = random.randint(0,2))
    labels[randnum + 12] = 1.0


    images = images.to(device)
    labels = torch.from_numpy(labels)
    latent = encoder(images, labels.to(device))
    latent = latent.cpu()

    #mean = latent.mean(dim=0)
    #print(mean)
    #std = (latent - mean).pow(2).mean(dim=0).sqrt()
    #print(std)

    # sample latent vectors from the normal distribution
    latent = torch.from_numpy(np.round(np.random.rand(128, d))).float()
    latent = torch.cat((latent, labels), dim=1)
    # reconstruct images from the random latent vectors
    latent = latent.to(device)
    img_recon = decoder(latent)
    img_recon = img_recon.cpu()

    fig, ax = plt.subplots(figsize=(40, 16.5))
    show_image(torchvision.utils.make_grid(img_recon[:100],10,5))
    plt.show()


TypeError: forward() missing 1 required positional argument: 'attributes'

In [ ]:
fig, ax = plt.subplots(10, 2, figsize=(10, 40))
fig.set_tight_layout(True)

c = 0
for data in testing_generator: # with "_" we just ignore the labels (the second element of the dataloader tuple)
    # Move tensor to the proper device
    if(c>=10):
        break
    image_batch = data["image"]


    #print("Random data shape", np.shape(random_data), "; data[labels] ", np.shape(data["labels"]))
    latent = data["labels"].float()
    latent = latent.to(device)
   
    
    img_recon = decoder(latent)

    img_recon = img_recon.detach().cpu().numpy()
    
    ax[c, 0].imshow(image_batch[0, 0, :, :])
    ax[c, 1].imshow(img_recon[0, 0, :, :])
    
    c += 1
    

plt.tight_layout()
    
    

In [ ]:
print(torch.__version__)